# Analysis of Chris Hughes' "Fair Shot" Plan

Analysis of cash transfer plan proposed by Chris Hughes in his 2018 book, [*FAIR SHOT: Rethinking Inequality and How We Earn*](http://www.fairshotbook.com/).

Per [Felix Salmon's review](https://www.nytimes.com/2018/02/25/books/review/chris-hughes-fair-shot.html) (summarized in my [Twitter thread](https://twitter.com/MaxGhenis/status/968216600670171136)), the elements of the plan are:

1) \$6k/year paid monthly to each adult in a household with income between \$6k-\$50k in previous year, or who earned <\$50k but (a) had cared for a dependent under age six or over age 70, or (b) were enrolled in an accredited college.

2) Households earning <\$6k in past year and who don't meet dependent or college requirements get their previous year's earnings matched 100%.

3) Funded by increasing to 50% the marginal rate on income and capital gains for households earning above \$250k.

Tax-Calculator CPS data ([documentation](http://open-source-economics.github.io/Tax-Calculator/)) lacks certain data elements necessary to fully implement this, so we make simplifying assumptions.

* **No ages of child dependents.** Instead flag tax units with at least one person under age 18 (`nu18 > 0`).
* **No ages of elderly parent dependents.** Instead flag all tax units that claimed an elderly parent as a dependent (`elderly_dependent`, which has [no age threshold](https://turbotax.intuit.com/tax-tips/family/steps-to-claiming-an-elderly-parent-as-a-dependent/L34jePeT9)).
* **No college enrollment**, so flag tax units with nonzero education tax credits (`c07230 > 0`).

Potential child care fields include:

* `e32800`: "Child/dependent-care expenses for qualifying persons from Form 2441"
* `f2441`: "number of child/dependent-care qualifying persons"
* `c07180`: "Credit for child and dependent care expenses from Form 2441"

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-03-01*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [2]:
tc.__version__

'0.16.1'

In [3]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Show one decimal in tables.
pd.set_option('precision', 2)

In [6]:
def gini(x, weights=None):
    if weights is None:
        weights = np.ones_like(x)
    # Calculate mean absolute deviation in two steps, for weights.
    count = np.multiply.outer(weights, weights)
    mad = np.abs(np.subtract.outer(x, x) * count).sum() / count.sum()
    # Gini equals half the relative mean absolute deviation.
    rmad = mad / np.average(x, weights=weights)
    return 0.5 * rmad